In [90]:
import mwclient
import logging
import bs4
import json
import requests
import urllib

import pygsheets
import pandas
import numpy

this api query out a json file of the references:

```https://www.gem.wiki/w/api.php?action=parse&page=East_African_Crude_Oil_Pipeline_(EACOP)&prop=externallinks&format=json```
https://www.gem.wiki/w/api.php?action=parse&page=Golden_Pass_LNG_Terminal&prop=externallinks&format=json

```https://www.gem.wiki/w/api.php?action=query&titles=East_African_Crude_Oil_Pipeline_(EACOP)&prop=extlinks&format=json``` doesn't seem comprehensive

which looks like this:
```
{
  "parse": {
    "title": "East African Crude Oil Pipeline (EACOP)",
    "pageid": 7391,
    "externallinks": [
      "http://bit.ly/2p697o3",
      "http://www.monitor.co.ug/Business/Commodities/Oil-pipeline-way-Uganda/-/688610/3105368/-/tdqa9lz/-/index.html",
      "https://www.reuters.com/article/ozabs-uk-uganda-tanzania-pipeline-idAFKBN2640EJ-OZABS",
      "http://www.monitor.co.ug/Business/Uganda--TZ-target-flexible-European-pipeline-funding/688322-4058602-fj2pi5/index.html",
      "https://www.pipelinesinternational.com/2020/05/21/total-acquires-uganda-pipeline-interests/",
      "https://eacop.com/",
      "https://e360.yale.edu/features/a-major-oil-pipeline-project-strikes-deep-at-the-heart-of-africa"
    ]
  }
}
```

In [91]:
gc = pygsheets.authorize(service_account_env_var='GDRIVE_API_CREDENTIALS')
spreadsheet = gc.open_by_key('1foPLE6K-uqFlaYgLPAUxzeXfDO5wOOqE7tibNHeqTek')

spreadsheet = gc.open_by_key('1foPLE6K-uqFlaYgLPAUxzeXfDO5wOOqE7tibNHeqTek')
gas_pipes = spreadsheet.worksheet('title', 'Gas pipelines').get_as_df(start='A3')
oil_pipes = spreadsheet.worksheet('title', 'Oil/NGL pipelines').get_as_df(start='A3')

pipes_orig_df = oil_pipes

# replace missing values -- with NaN
pipes_orig_df.replace('--', numpy.nan, inplace=True)

/var/folders/fl/t07mc8053p33mn6mdmvp45580000gn/T/ipykernel_19470/3153225262.py:11: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  pipes_orig_df.replace('--', numpy.nan, inplace=True)


In [92]:
# check that for any pipelines with wiki pages, the pipeline name matches the wiki page
url_start = 'https://www.gem.wiki/'
pipes_orig_df_with_wikis = pipes_orig_df.loc[pipes_orig_df.Wiki!='']
for idx,row in pipes_orig_df_with_wikis.iterrows():
    pipeline_name = row.PipelineName
    wiki = row.Wiki
    pipeline_name_wiki = pipeline_name.replace(' ', '_')
    pipeline_name_wiki = pipeline_name_wiki.replace('–', '%e2%80%93')
    pipeline_name_wiki = pipeline_name_wiki.replace("'", '%27')
    pipeline_name_wiki = pipeline_name_wiki.replace("í", '%c3%ad')
    pipeline_name_wiki = pipeline_name_wiki.replace('ı', '%C4%B1')
    pipeline_name_wiki = pipeline_name_wiki.replace('á', '%C3%A1')
    pipeline_name_wiki = pipeline_name_wiki.replace('á', '%C3%B3')
    pipeline_name_wiki = pipeline_name_wiki.replace('ñ', '%c3%b1')
    pipeline_name_wiki = pipeline_name_wiki.replace('ü', '%C3%9C')
    if wiki != url_start+pipeline_name_wiki:
        print(wiki)
        print(url_start+pipeline_name_wiki)
        print()

https://www.gem.wiki/Ronier%E2%80%93Djermaya_Oil_Pipeline
https://www.gem.wiki/Ronier%e2%80%93Djermaya_Oil_Pipeline

https://www.gem.wiki/Chad%E2%80%93Cameroon_Oil_Pipeline
https://www.gem.wiki/Chad%e2%80%93Cameroon_Oil_Pipeline

https://www.gem.wiki/Uganda%E2%80%93Kenya_Crude_Oil_Pipeline_(UKCOP)
https://www.gem.wiki/Uganda%e2%80%93Kenya_Crude_Oil_Pipeline_(UKCOP)

https://www.gem.wiki/Bab-Habshan%E2%80%93Fujairah_Oil_Pipeline
https://www.gem.wiki/Bab-Habshan%e2%80%93Fujairah_Oil_Pipeline

https://www.gem.wiki/Oman_Main_Line%E2%80%93Ras_Markaz_Oil_Pipeline
https://www.gem.wiki/Oman_Main_Line%e2%80%93Ras_Markaz_Oil_Pipeline

https://www.gem.wiki/Bab-Habshan%E2%80%93Jebel_Dhanna_Oil_Pipeline
https://www.gem.wiki/Bab-Habshan%e2%80%93Jebel_Dhanna_Oil_Pipeline

https://www.gem.wiki/Kazakhstan%E2%80%93China_Oil_Pipeline
https://www.gem.wiki/Kazakhstan%e2%80%93China_Oil_Pipeline

https://www.gem.wiki/Eastern_Siberia%E2%80%93Pacific_Ocean_Oil_Pipeline
https://www.gem.wiki/Eastern_Siberia%e2%8

In [70]:
pipeline_name = 'Alberta Clipper Oil Pipeline'

In [71]:
pipeline_name = pipeline_name.replace(' ', '_')
pipeline_name = pipeline_name.replace('–', '%E2%80%93')

In [72]:
pipeline_name

'Alberta_Clipper_Oil_Pipeline'

In [65]:
urllib.parse.quote('Alberta Clipper Oil Pipeline')

'Alberta%20Clipper%20Oil%20Pipeline'

In [33]:
requests.get('https://www.gem.wiki/w/api.php?action=parse&page=East_African_Crude_Oil_Pipeline_(EACOP)&prop=externallinks&format=json').json()['parse']['externallinks'].__len__()

62

In [33]:
requests.get('https://www.gem.wiki/w/api.php?action=parse&page=East_African_Crude_Oil_Pipeline_(EACOP)&prop=externallinks&format=json').json()['parse']['externallinks'].__len__()

62

## check for existence of wiki pages, fix missing ones

In [ ]:
# for each row, loop through and check whether a wiki page exists for it
# check if there are any # in the wiki pages
# spit out all the rows that DON'T have a wiki page associated with them (will prob just be poorly formatted)

## then loop though and get json data out

In [34]:
# get json text of externallinks
# if globalenergymonitor.org is in it, remove from list
# save json as projectid file
# save spreadsheet of a big long joined string with ProjectID

In [36]:
request_json = requests.get('https://www.gem.wiki/w/api.php?action=parse&page=East_African_Crude_Oil_Pipeline_(EACOP)&prop=externallinks&format=json').json()

In [41]:
request_json['parse']['externallinks'].append(gem_site_string)
request_json['parse']['externallinks'].append(gem_site_string)

In [42]:
request_json['parse']['externallinks']

['http://bit.ly/2p697o3',
 'http://www.monitor.co.ug/Business/Commodities/Oil-pipeline-way-Uganda/-/688610/3105368/-/tdqa9lz/-/index.html',
 'https://www.reuters.com/article/ozabs-uk-uganda-tanzania-pipeline-idAFKBN2640EJ-OZABS',
 'http://www.monitor.co.ug/Business/Uganda--TZ-target-flexible-European-pipeline-funding/688322-4058602-fj2pi5/index.html',
 'https://www.pipelinesinternational.com/2020/05/21/total-acquires-uganda-pipeline-interests/',
 'https://eacop.com/',
 'https://eacop.com/overview/',
 'http://allafrica.com/stories/201708070048.html',
 'http://www.theeastafrican.co.ke/news/Museveni-visit-to-Dar-rescues-oil-pipeline-deal/2558-3838726-mrxw7xz/index.html',
 'https://www.ft.com/content/597a2b01-fb54-4fd3-b326-dadf52dc250a',
 'https://www.stopeacop.net/whos-behind-eacop',
 'https://www.total.com/media/news/press-releases/Uganda_Tanzania-final-agreements-for-lake-albert-resources-development-project',
 'https://allafrica.com/stories/202108040210.html',
 'https://www.ogj.com/pi

In [43]:
gem_site_string = 'http://globalenergymonitor.org'

# do it iteratively in case there's more than 1
while gem_site_string in request_json['parse']['externallinks']:
    request_json['parse']['externallinks'].remove(gem_site_string)

In [21]:
page = site.pages['East African Crude Oil Pipeline (EACOP)']

In [24]:
page_text = page.text()

In [26]:
bs4.BeautifulSoup(page_text)

{{Navbar-FossilTracker}}
'''East African Crude Oil Pipeline (EACOP)''', also known as '''Uganda–Tanzania Crude Oil Pipeline''' or '''Hoima–Tanga Port Oil Pipeline''', is a proposed oil pipeline in East Africa.<ref name="source1">[http://bit.ly/2p697o3 Uganda–Tanzania Crude Oil Pipeline,] Wikipedia, accessed September 2017</ref>

==Location==
The oil pipeline would start in Buseruka sub-county, Hoima District, in Uganda's Western Region. It would travel in a general south-easterly direction to pass through Masaka in Uganda, Bukoba in Tanzania, loop around the southern shores of Lake Victoria, continue through Shinyanga and Singida town, and end at the Port of Tanga, Tanzania, for export from the Indian Ocean.<ref name="Which">{{cite web | accessdate=7 March 2016 | url=http://www.monitor.co.ug/Business/Commodities/Oil-pipeline-way-Uganda/-/688610/3105368/-/tdqa9lz/-/index.html | title=Oil pipeline: Which way for Uganda? | date=7 March 2016 | first=Frederic| last=Musisi  | newspaper=[[Dai

In [ ]:
# pull out and list everything between a <ref></ref> tag
https://en.wikipedia.org/w/api.php?action=parse&page=Coffee&prop=externallinks
https://gem.wiki/w/api.php?action=parse&page=East_African_Crude_Oil_Pipeline_(EACOP)&prop=externallinks

In [3]:
site = mwclient.Site('www.gem.wiki')

In [4]:
site

<Site object 'www.gem.wiki/w/'>